In [8]:
from datasets import load_dataset

eli5= load_dataset("eli5", split="train_asks[:5000]") 

Found cached dataset eli5 (C:/Users/AIXI/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


In [9]:
eli5= eli5.train_test_split(test_size=0.2)

In [10]:
eli5["test"][22]

{'q_id': '2fhvmo',
 'title': 'Can somebody please explain the difference between a shear thinning and a thixotropic fluid?',
 'selftext': "As far as i'm aware a shear thinning fluid is one that shows a decrease in viscosity with increasing shear stress.\n\nHaving looked up the definition of a thixotropic fluid, it seems as though they're the same thing, although from what I've read is seems as though they do a have a subtly different meaning.\n\nCould anyone shed some light on what the difference between these two things is? Thanks.",
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['ck9sk4e'],
  'text': ["I think your hangup is not so much that they are very similar, but rather that they are describing **ideal** behaviors that don't really exist independently from each other in real life materials that we know of.\n\nShear-thinning/thickening is just a [nonlinear relationship between shear stress and shear strain](_URL_1_); thins when sheared. Simple enough. Thixotro

In [11]:
# eli5= eli5.flatten()
# eli5["train"][22]

In [32]:
# for i in eli5["train"]:
#     print(i)

In [33]:
# import pandas as pd

def process_dataset(datasett):
    # list to store extracted elements
    titles = []
    selftexts = []
    texts = []

    for item in datasett:
       title = item['title']
       selftext = item['selftext']
       text = item['answers']['text'][0]

       titles.append(title)
       selftexts.append(selftext)
       texts.append(text)

    df = pd.DataFrame({
       'Title': titles,
       'Selftext': selftexts,
       'Text': texts
    })

    return df

In [34]:
df_train= process_dataset(eli5['train'])
df_test= process_dataset(eli5['test'])

In [35]:
print(len(df_train))
print(len(df_test))

4000
1000


In [37]:
df_test.head()

,Title,Selftext,Text
0,[Earth Sciences] What are the geological speci...,My searching of the internet and reddit hasn't...,It's tough to say the exact method of formatio...
1,How come different allotropes of an element ha...,"I have never studied chemistry, but I know tha...",The carbon atoms in diamond and graphite bond ...
2,Is there a scientific explanation for people w...,After digging through /r/deadbedrooms and /r/l...,Although this is probably incredibly more comp...
3,Ligo gravitational observatory and nuclear bombs?,Could a nuclear bomb trigger ligo? Would we kn...,Yes it would trigger ligo but they back check ...
4,Different ethnicity and their comfort levels i...,I have a diverse group of friends. Trinidadia...,Why would it have to be genetic? Maybe they ju...


In [38]:
## preprocessing of EVAL dataset
eval_text_data= df_test['Text'].tolist()
tokenized_eval_data= tokenizer(eval_text_data, truncation=True, padding=True)

# creating a TextDataset for evaluation
eval_dataset= TextDataset(tokenizer=tokenizer, file_path=None, column_names= ['input_ids', 'attention_mask'], data=tokenized_eval_data)

NameError: name 'tokenizer' is not defined

In [40]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling

model_name= "distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model= AutoModelForCausalLM.from_pretrained(model_name)

texts= df_train["Text"].tolist()

# Add a padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

tokenized_data = tokenizer(texts, truncation=True, padding=True, return_tensors="pt", max_length=128)

# Create a TextDataset
dataset = TextDataset(tokenizer=tokenizer, file_path=None, column_names=['input_ids', 'attention_mask'], data=tokenized_data)

# Create a DataCollator for Language Modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\AIXI\AppData\Local\Temp\ipykernel_8380\1438674355.py:16 in <module>                     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\AIXI\\AppData\\Local\\Temp\\ipykernel_8380\\1438674355.py'                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: TextDataset.__init__() got an unexpected keyword argument 'column_names'

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=eval_dataset["test"],
    data_collator=data_collator,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

trainer.train()

In [41]:
import torch
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

# Initialize the GPT-2 tokenizer and model
model_name = 'distilgpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add a padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize your text data
text_data = df_train['Text'].tolist()
tokenized_data = tokenizer(text_data, truncation=True, padding='max_length', return_tensors='pt', max_length=128)

# Tokenize evaluation data
eval_text_data = df_test['Text'].tolist()
tokenized_eval_data = tokenizer(eval_text_data, truncation=True, padding='max_length', return_tensors='pt', max_length=128)

# Create datasets using the datasets library
train_dataset = {'input_ids': tokenized_data['input_ids'], 'attention_mask': tokenized_data['attention_mask']}
eval_dataset = {'input_ids': tokenized_eval_data['input_ids'], 'attention_mask': tokenized_eval_data['attention_mask']}

# Create a DataCollator for Language Modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./fine-tuned-gpt2',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=10_000,
    do_eval=True,
    logging_steps=100,
    evaluation_report=True,
    max_seq_length=128,
)

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Add the evaluation dataset here
    compute_metrics=None,  # Optional: Define custom evaluation metrics
    tokenizer=tokenizer,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model()

In [42]:
torch.cuda.is_available()

True

In [76]:
# def preprocess_function(examples, tokenizer):
#     return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [77]:
# tokenized_eli5= eli5.map(
#     preprocess_function(eli5["train"], tokenizer),
#     # preprocess_function,
#     batched=False,
#     # num_proc=4,
#     remove_columns=eli5["train"].column_names,
# )

In [78]:
# block_size = 128


# def group_texts(examples):
#     # Concatenate all texts.
#     concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
#     total_length = len(concatenated_examples[list(examples.keys())[0]])
#     # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
#     # customize this part to your needs.
#     if total_length >= block_size:
#         total_length = (total_length // block_size) * block_size
#     # Split by chunks of block_size.
#     result = {
#         k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
#         for k, t in concatenated_examples.items()
#     }
#     result["labels"] = result["input_ids"].copy()
#     return result

In [79]:
# lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

In [40]:
eli5.flatten()

DatasetDict({
    train: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers.a_id', 'answers.text', 'answers.score', 'title_urls.url', 'selftext_urls.url', 'answers_urls.url'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers.a_id', 'answers.text', 'answers.score', 'title_urls.url', 'selftext_urls.url', 'answers_urls.url'],
        num_rows: 1000
    })
})